In [3]:
import os
from PIL import Image

# 이미지 크기로 비어있는 좌표를 채우는 함수
def fill_or_label_images(dataset_path, output_txts_path):
    # 모든 폴더를 탐색
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            # 이미지 파일만 처리
            if file.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, file)
                image_name, _ = os.path.splitext(file)
                
                # properties 파일 경로
                properties_file = os.path.join(root, 'crop_area.properties')
                
                # 이미지 크기 확인
                try:
                    with Image.open(image_path) as img:
                        width, height = img.size
                except Exception as e:
                    print(f"Error opening image {image_name}: {e}")
                    continue
                
                # 기본 좌표 (이미지 전체 크기)
                default_coords = [0, 0, width, height]
                
                filled_coords = default_coords
                
                # properties 파일에서 좌표를 불러옴
                if os.path.exists(properties_file):
                    with open(properties_file, 'r', encoding='utf-8') as f:
                        lines = f.readlines()
                        for line in lines:
                            if image_name in line:
                                coords = line.split('=')[-1].strip().split(',')
                                if len(coords) == 4:
                                    # 각 좌표가 비어있지 않으면 그 값 사용, 비어있으면 기본 값 사용
                                    filled_coords = [
                                        coords[0] if coords[0] != '' else default_coords[0],
                                        coords[1] if coords[1] != '' else default_coords[1],
                                        coords[2] if coords[2] != '' else default_coords[2],
                                        coords[3] if coords[3] != '' else default_coords[3]
                                    ]
                                break
                
                # filled_coords에서 좌표가 4개가 있는지 확인
                if len(filled_coords) == 4:
                    try:
                        # YOLO 형식으로 변환
                        x_center = (int(filled_coords[0]) + int(filled_coords[2])) / (2 * width)
                        y_center = (int(filled_coords[1]) + int(filled_coords[3])) / (2 * height)
                        bbox_width = (int(filled_coords[2]) - int(filled_coords[0])) / width
                        bbox_height = (int(filled_coords[3]) - int(filled_coords[1])) / height
                        
                        # 출력 경로 설정 및 YOLO 형식으로 저장
                        label_file_path = os.path.join(output_txts_path, f"{image_name}.txt")
                        os.makedirs(output_txts_path, exist_ok=True)
                        
                        with open(label_file_path, 'w') as label_file:
                            label_file.write(f"0 {x_center} {y_center} {bbox_width} {bbox_height}\n")
                    
                    except Exception as e:
                        print(f"Error processing label for {image_name}: {e}")
                else:
                    print(f"Skipping {image_name}, insufficient coordinates.")

dataset_path = 'dataset/kfood'
output_txts_path = 'dataset/empty_labeling'

# 라벨링 함수 실행
fill_or_label_images(dataset_path, output_txts_path)


c:\Users\admin\miniconda3\envs\ds_study\Lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [1]:
import os

# 경로 설정
empty_labeling_path = 'dataset/empty_labeling'

# 라벨 파일 수정 함수
def fix_labels(empty_labeling_path):
    for label_file in os.listdir(empty_labeling_path):
        if label_file.endswith('.txt'):
            label_path = os.path.join(empty_labeling_path, label_file)
            
            # 파일 이름에서 중간 숫자를 추출하여 클래스 ID로 사용
            file_name = os.path.splitext(label_file)[0]
            middle_number = file_name.split('_')[1]  # 중간의 숫자 추출
            class_id = int(middle_number)  # 숫자를 클래스 ID로 변환

            # 라벨 파일 읽기
            with open(label_path, 'r') as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()
                coords = [float(p) for p in parts[1:]]  # 좌표 부분 추출

                # 음수 좌표는 0으로 수정
                coords = [max(0, min(c, 1)) for c in coords]

                new_line = f"{class_id} " + " ".join(map(str, coords)) + "\n"
                new_lines.append(new_line)

            # 수정된 라벨 파일 저장
            with open(label_path, 'w') as f:
                f.writelines(new_lines)

# 라벨 수정 실행
fix_labels(empty_labeling_path)

print("라벨 파일 수정이 완료되었습니다.")

라벨 파일 수정이 완료되었습니다.
